# Collecting the relevant data and putting it all together
This notebook goes through the process of retrieving the relevant data we need for our data story about profanity in music. In order to analyse profanity in music we want to gather the following information about as many songs as possible:

*   Song name
*   Artist name
*   Song release year
*   Song duration
*   Number of words in the song
*   Number of profane words in the song
*   Song genre

To do so, we will be using the [million song dataset](http://millionsongdataset.com/) by Thierry Bertin-Mahieux, Daniel P.W. Ellis, Brian Whitman, Paul Lamere from Columbia University and The Echo Nest. This dataset includes metadata about a million songs from the last century, up to 2011.



In [ ]:
import sqlite3
import os
import pandas as pd
import numpy as np

## Getting song metadata from the original million song dataset
Initially, we thought we had to access the entire 300GB dataset to gather song data about all one million songs. Thus, we initially used a subset of 10 000 songs (1.8GB) to develop the code below. Then, by chance we found that the million song dataset website also supplied a SQLite database file containing a limited number of fields for all one million songs, being just 712 MB in size.

As we are only interested in a subset of the several fields supplied in the dataset, we will extract the relevant fields into a new dataset. Not all fields we are interested in are included in the original million song dataset (spefically lyric data and genre data is not included), so this information will be added using extensions to the dataset through joining of tables in later sections below.

### Downloading the data

In [ ]:
!wget http://millionsongdataset.com/sites/default/files/AdditionalFiles/track_metadata.db

--2020-04-24 10:39:23--  http://millionsongdataset.com/sites/default/files/AdditionalFiles/track_metadata.db
Resolving millionsongdataset.com (millionsongdataset.com)... 173.231.209.32
Connecting to millionsongdataset.com (millionsongdataset.com)|173.231.209.32|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 746180608 (712M)
Saving to: ‘track_metadata.db’

track_metadata.db   100%[===================>] 711.61M  15.3MB/s    in 49s     

2020-04-24 10:40:13 (14.6 MB/s) - ‘track_metadata.db’ saved [746180608/746180608]



### Retrieving the relevant fields

In [ ]:
conn = sqlite3.connect("/content/track_metadata.db")

In [ ]:
# We execute this query to get a sense of how the database is laid out (table names, field names)
res = conn.execute("SELECT * FROM sqlite_master WHERE type='table'")
res.fetchall()

[('table',
  'songs',
  'songs',
  2,
  'CREATE TABLE songs (track_id text PRIMARY KEY, title text, song_id text, release text, artist_id text, artist_mbid text, artist_name text, duration real, artist_familiarity real, artist_hotttnesss real, year int, track_7digitalid int, shs_perf int, shs_work int)')]

In [ ]:
# build the SQL query to retrieve the desired columns
columns = ["track_id", "artist_name", "title", "year", "duration"]
query = f"SELECT {', '.join(columns)} FROM songs"

# query the database
res = conn.execute(query)
selected_fields = res.fetchall()


print(f"found {len(selected_fields)} rows")

found 1000000 rows


In [ ]:
# Some sample data
for k in range(3500, 3515):
  print(list(selected_fields)[k])

('TRMGZHK12903D0A31E', 'Hip Hop Pantsula', 'Ancestors', 0, 282.77506)
('TRMGZUS128F4262A57', 'The Aluminum Group', 'Headphones', 2008, 206.62812)
('TRMGZAU12903CBAA65', 'Groove Generator', "Let's do this", 0, 197.27628)
('TRMGZTZ128F930243B', 'Cricco Castelli', 'Desperate Situation', 0, 306.96444)
('TRMGZXZ128F4288352', 'Johnny Hallyday', 'La Quête - Vieilles Charrues 2006', 0, 199.47057)
('TRMGZDG128F425DB7B', 'Kilpi', 'Järjen Riemuvoitto', 2008, 298.65751)
('TRMGZIU128F93024A3', 'Rey Ruiz', 'Vereda Tropical', 1996, 244.06159)
('TRMGZUE128F427C14D', 'Vicente Amigo', 'Querido Metheny', 1995, 569.83465)
('TRMGZQD12903CB7FD2', 'Charlie Parker', "Bird's Nest", 1993, 164.85832)
('TRMGZMD128F1496EA1', 'Envoy', 'Elektra', 0, 434.75546)
('TRMGZBO12903CB2735', 'Joyce Sims', '(I\x19m Back) I Love You More', 0, 310.56934)
('TRMGZTZ128F146EF17', 'Un pinguino en mi ascensor', 'Atrapados En El Ascensor', 1988, 198.89587)
('TRMGZHA128F423E750', 'Dynamite Deluxe', 'So Laut Es Geht', 2008, 165.69424)


### Generate pandas dataframe
We create pandas dataframes of the data to make it easy to join with the other data sources later.


In [ ]:
msd_metadata_df = pd.DataFrame(selected_fields, columns=columns)
msd_metadata_df

,track_id,artist_name,title,year,duration
0,TRMMMYQ128F932D901,Faster Pussy cat,Silent Night,2003,252.05506
1,TRMMMKD128F425225D,Karkkiautomaatti,Tanssi vaan,1995,156.55138
2,TRMMMRX128F93187D9,Hudson Mohawke,No One Could Ever,2006,138.97098
3,TRMMMCH128F425532C,Yerba Brava,Si Vos Querés,2003,145.05751
4,TRMMMWA128F426B589,Der Mystic,Tangle Of Aspens,0,514.29832
...,...,...,...,...,...
999995,TRYYYUS12903CD2DF0,Kiko Navarro,O Samba Da Vida,0,217.44281
999996,TRYYYJO128F426DA37,Kuldeep Manak,Jago Chhadeo,0,244.16608
999997,TRYYYMG128F4260ECA,Gabriel Le Mar,Novemba,0,553.03791
999998,TRYYYDJ128F9310A21,Elude,Faraday,0,484.51873


## Retrieving a list of profane words
We will use Facebook's list of bad words from [freewebheaders.com](https://www.freewebheaders.com/bad-words-list-and-page-moderation-words-list-for-facebook/)

In [ ]:
!wget https://www.freewebheaders.com/download/files/facebook-bad-words-list_comma-separated-text-file_2018_07_29.zip

--2020-04-24 10:40:18--  https://www.freewebheaders.com/download/files/facebook-bad-words-list_comma-separated-text-file_2018_07_29.zip
Resolving www.freewebheaders.com (www.freewebheaders.com)... 104.18.34.100, 104.18.35.100, 2606:4700:3036::6812:2364, ...
Connecting to www.freewebheaders.com (www.freewebheaders.com)|104.18.34.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6310 (6.2K) [application/zip]
Saving to: ‘facebook-bad-words-list_comma-separated-text-file_2018_07_29.zip’

facebook-bad-words- 100%[===================>]   6.16K  --.-KB/s    in 0s      

2020-04-24 10:40:19 (23.5 MB/s) - ‘facebook-bad-words-list_comma-separated-text-file_2018_07_29.zip’ saved [6310/6310]



In [ ]:
!unzip '/content/facebook-bad-words-list_comma-separated-text-file_2018_07_29.zip'

Archive:  /content/facebook-bad-words-list_comma-separated-text-file_2018_07_29.zip
  inflating: facebook-bad-words-list_comma-separated-text-file_2018_07_29.txt  


In [ ]:
with open('/content/facebook-bad-words-list_comma-separated-text-file_2018_07_29.txt', encoding="utf8", errors="ignore") as file:
  profane_words = file.readlines()[-1].split(", ")

print(len(profane_words))
profane_words[711]

1703


'fuck'

## Getting lyrics data for the million song dataset from musiXmatch
Conveniently, the million song dataset team published an extension to the original dataset which includes [song lyric data from musiXmatch](http://millionsongdataset.com/musixmatch/). This allows for joining of lyrics data to the above extracted metadata as the same song identifiers are used in both datasets.

See readme [here](https://github.com/tbertinmahieux/MSongsDB/tree/master/Tasks_Demos/Lyrics
) for description of the musiXmatch database. It contains a table where each row is the number of occurences of a given word in a given song from the million song dataset.

The goal is to generate a row per song which includes `song_id`, `number_of_words`, `number_of_profane_words`. Which can then be joined with the million song dataset file.

### Getting the data

In [ ]:
!wget http://millionsongdataset.com/sites/default/files/AdditionalFiles/mxm_dataset.db

--2020-04-24 10:40:24--  http://millionsongdataset.com/sites/default/files/AdditionalFiles/mxm_dataset.db
Resolving millionsongdataset.com (millionsongdataset.com)... 173.231.209.32
Connecting to millionsongdataset.com (millionsongdataset.com)|173.231.209.32|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2441556992 (2.3G)
Saving to: ‘mxm_dataset.db’

mxm_dataset.db      100%[===================>]   2.27G  15.6MB/s    in 2m 39s  

2020-04-24 10:43:04 (14.6 MB/s) - ‘mxm_dataset.db’ saved [2441556992/2441556992]



In [ ]:
conn = sqlite3.connect("/content/mxm_dataset.db")



### Counting profane words

In [ ]:
# SQL query for summing up all profane words in a song
query = f"SELECT track_id, SUM(case when word IN {tuple(profane_words)} then count else 0 end) AS profane_count FROM lyrics GROUP BY track_id"


# query the database
res = conn.execute(query)
profane_count = res.fetchall()


print(f"found {len(profane_count)} rows")

found 237662 rows


In [ ]:
profane_count[1020:1030] # Sample data

[('TRACSIR12903CE81A8', 0),
 ('TRACSJC128EF3432F8', 0),
 ('TRACSKH128F934020B', 8),
 ('TRACSNO128F424F331', 0),
 ('TRACSQN12903CDCB38', 0),
 ('TRACSSJ128F92E195B', 0),
 ('TRACSTF128F4238A87', 0),
 ('TRACSYZ12903CF3C93', 0),
 ('TRACTEB128F4264605', 1),
 ('TRACTGL128F425BEFC', 6)]

In [ ]:
# Check a specific track to see that the aggregation is correct
q = f"SELECT track_id, word, count FROM lyrics WHERE word IN {tuple(profane_words)} AND track_id='TRACSKH128F934020B'"
res = conn.execute(q)
profane_count_specific_song = res.fetchall()
profane_count_specific_song

[('TRACSKH128F934020B', 'god', 4),
 ('TRACSKH128F934020B', 'fuck', 2),
 ('TRACSKH128F934020B', 'kill', 1),
 ('TRACSKH128F934020B', 'murder', 1)]

### Counting all words per song

In [ ]:
# SQL query for summing up all words in a song
query = "SELECT track_id, SUM(count) AS word_count FROM lyrics GROUP BY track_id"


# query the database
res = conn.execute(query)
word_count = res.fetchall()


print(f"found {len(word_count)} rows")

found 237662 rows


In [ ]:
word_count[1020:1030] # Sample data

[('TRACSIR12903CE81A8', 296),
 ('TRACSJC128EF3432F8', 183),
 ('TRACSKH128F934020B', 243),
 ('TRACSNO128F424F331', 184),
 ('TRACSQN12903CDCB38', 149),
 ('TRACSSJ128F92E195B', 206),
 ('TRACSTF128F4238A87', 160),
 ('TRACSYZ12903CF3C93', 191),
 ('TRACTEB128F4264605', 54),
 ('TRACTGL128F425BEFC', 384)]

### Collecting word count and profane count in one query

In [ ]:
# SQL query combining the above queries into one.
query = f"SELECT track_id, SUM(count) AS word_count, SUM(case when word IN {tuple(profane_words)} then count else 0 end) AS profane_count"
query += " FROM lyrics GROUP BY track_id"


# query the database
res = conn.execute(query)
word_profane_count = res.fetchall()


print(f"found {len(word_profane_count)} rows")

found 237662 rows


In [ ]:
word_profane_count[1020:1030] # Sample data

[('TRACSIR12903CE81A8', 296, 0),
 ('TRACSJC128EF3432F8', 183, 0),
 ('TRACSKH128F934020B', 243, 8),
 ('TRACSNO128F424F331', 184, 0),
 ('TRACSQN12903CDCB38', 149, 0),
 ('TRACSSJ128F92E195B', 206, 0),
 ('TRACSTF128F4238A87', 160, 0),
 ('TRACSYZ12903CF3C93', 191, 0),
 ('TRACTEB128F4264605', 54, 1),
 ('TRACTGL128F425BEFC', 384, 6)]

### Generate pandas dataframe

In [ ]:
mxm_df = pd.DataFrame(word_profane_count, columns=['track_id', 'word_count', 'profane_count'])
mxm_df

,track_id,word_count,profane_count
0,TRAAAAV128F421A322,103,1
1,TRAAABD128F429CF47,226,0
2,TRAAAED128E0783FAB,421,0
3,TRAAAEF128F4273421,139,0
4,TRAAAEW128F42930C0,115,0
...,...,...,...
237657,TRZZZXA128F428ED56,163,3
237658,TRZZZXV128F4289747,207,0
237659,TRZZZYV128F92E996D,334,4
237660,TRZZZYX128F92D32C6,297,0


## Retrieving genre data from the last.fm extension
In order to retrieve genre data, we will be using the [last.fm extension](http://millionsongdataset.com/lastfm/) of the million song database. The extension includes "tags" from last.fm for specific songs, which among other things should include genre data. Again, this dataset uses the same song identifiers as the others above, allowing us to join genre data to the above datasets.

 In order to understand how to use the SQLite db, we had a look at [this demo](http://millionsongdataset.com/sites/default/files/lastfm/demo_tags_db.py).

### Downloading the data

In [ ]:
!wget http://millionsongdataset.com/sites/default/files/lastfm/lastfm_tags.db

--2020-04-24 10:48:48--  http://millionsongdataset.com/sites/default/files/lastfm/lastfm_tags.db
Resolving millionsongdataset.com (millionsongdataset.com)... 173.231.209.32
Connecting to millionsongdataset.com (millionsongdataset.com)|173.231.209.32|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 594083840 (567M)
Saving to: ‘lastfm_tags.db’

lastfm_tags.db      100%[===================>] 566.56M  16.1MB/s    in 37s     

2020-04-24 10:49:25 (15.2 MB/s) - ‘lastfm_tags.db’ saved [594083840/594083840]



In [ ]:
conn = sqlite3.connect("/content/lastfm_tags.db")

### Retrieve list of top 100 tags
We will select the top 100 most popular tags to get a broad arrangement of genres to choose from in our analysis. The list of tags sorted by popularity can be found [here](http://millionsongdataset.com/sites/default/files/lastfm/lastfm_unique_tags.txt).

In [ ]:
!wget http://millionsongdataset.com/sites/default/files/lastfm/lastfm_unique_tags.txt

--2020-04-24 10:49:27--  http://millionsongdataset.com/sites/default/files/lastfm/lastfm_unique_tags.txt
Resolving millionsongdataset.com (millionsongdataset.com)... 173.231.209.32
Connecting to millionsongdataset.com (millionsongdataset.com)|173.231.209.32|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10115638 (9.6M) [text/plain]
Saving to: ‘lastfm_unique_tags.txt’

lastfm_unique_tags. 100%[===================>]   9.65M  5.22MB/s    in 1.8s    

2020-04-24 10:49:29 (5.22 MB/s) - ‘lastfm_unique_tags.txt’ saved [10115638/10115638]



In [ ]:
with open('/content/lastfm_unique_tags.txt') as file:
  genres = file.readlines()[0:100]

genres = list(map(lambda item: item.split('\t')[0], genres))

genres[:10]


['rock',
 'pop',
 'alternative',
 'indie',
 'electronic',
 'female vocalists',
 'favorites',
 'Love',
 'dance',
 '00s']

### Retrieving songs and correlated tags

In [ ]:
# Get an understanding of the db tables and columns
res = conn.execute("SELECT * FROM sqlite_master WHERE type='table'")
res.fetchall()

[('table', 'tags', 'tags', 2, 'CREATE TABLE tags (tag TEXT)'),
 ('table', 'tids', 'tids', 3, 'CREATE TABLE tids (tid TEXT)'),
 ('table',
  'tid_tag',
  'tid_tag',
  4,
  'CREATE TABLE tid_tag (tid INT, tag INT, val FLOAT)')]

In [ ]:
res = conn.execute(f"SELECT tids.tid, tags.tag FROM tid_tag, tags, tids WHERE tags.ROWID=tid_tag.tag AND tids.ROWID=tid_tag.tid and tags.tag IN {tuple(genres)}")
tracks_with_tags = res.fetchall()

In [ ]:
len(tracks_with_tags)

1929985

In [ ]:
tracks_with_tags[50000:50005]

[('TRBFUMI128F42B5731', '60s'),
 ('TRNAGPR12903CDF6F6', '60s'),
 ('TRSMWMH128F145A86B', '60s'),
 ('TRSRYGJ128EF345FFD', '60s'),
 ('TRSSKLW128F92EF99C', '60s')]

### Generate pandas dataframe

In [ ]:
lastfm_df = pd.DataFrame(tracks_with_tags, columns=['track_id', 'genre'])
lastfm_df.head()

,track_id,genre
0,TRCJLLZ128F93239CA,00s
1,TRCIIUA128F93652FB,00s
2,TRCODME128F930F717,00s
3,TRCOTZQ128F42AE3C0,00s
4,TRCRLFH128F9348F4B,00s


## Joining it all together
Now lets join all the dataframes together to one big nice dataframe. We will be using inner joins so that we retain only the songs for which we have metadata, lyrics data and genre data.

In [ ]:
metadata_lyrics = msd_metadata_df.join(mxm_df.set_index('track_id'), on='track_id', how="inner")
full_df = metadata_lyrics.join(lastfm_df.set_index('track_id'), on="track_id", how="inner")


In [ ]:
full_df

,track_id,artist_name,title,year,duration,word_count,profane_count,genre
15,TRMMMKI128F931D80D,Lena Philipsson,006,1998,262.26893,246,0,pop
16,TRMMMUT128F42646E8,Shawn Colvin,(Looking For) The Heart Of Saturday,1994,216.47628,328,2,cover
17,TRMMMQY128F92F0EA3,Dying Fetus,Ethos of Coercion,2009,196.02240,78,0,death metal
17,TRMMMQY128F92F0EA3,Dying Fetus,Ethos of Coercion,2009,196.02240,78,0,metal
18,TRMMMTK128F424EF7C,Emery,Rock-N-Rule,2007,217.57342,246,1,00s
...,...,...,...,...,...,...,...,...
999964,TRYYYCH128F149840B,Something Corporate,If You C Jordan,2001,258.66404,259,0,seen live
999979,TRYYYFV128F4277D53,Starflyer 59,Too Much Fun,1998,462.26240,53,0,alternative rock
999980,TRYYYWS128EF34A1FF,Stevie Wonder,Think Of Me As Your Soldier,1971,219.14077,165,0,soul
999992,TRYYYZM128F428E804,SKYCLAD,Inequality Street,1996,241.21424,345,2,folk


In [ ]:
len(full_df["track_id"].unique())

143847

In [ ]:
list(full_df.columns)

['track_id',
 'artist_name',
 'title',
 'year',
 'duration',
 'word_count',
 'profane_count',
 'genre']

### Write data to csv
We finish off by writing the complete dataset to csv in order to export it to Tableau for analysis.

In [ ]:
full_df.to_csv("msd_full.csv", index=False)

# Appendix
Here we saved some code we made that eventually became redundant/unnecessary.

In [ ]:
def write_data_to_csv(filename, columns, data, separator=","):
'''
  export data to csv-file.

  filename: string filename - file will be overwritten
  columns: list of strings, names of the columns
  data: list of data, each item in the list should be one row
  separator: optional string that should be used as separator
'''

  with open(filename, "w") as write_file:
      write_file.write(separator.join(columns) +'\n')
      for row in data:
          writeRow = separator.join([str(i) for i in row]) + "\n"
          write_file.write(writeRow)